In [24]:
import pymysql
import csv

In [16]:
user = 'root'
passwd = ''

In [32]:
# 创建连接
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd=passwd, db='myemployees', charset='utf8')

In [33]:
cursor = conn.cursor(cursor=pymysql.cursors.DictCursor)
# 以字典表形式返回

In [30]:
conn.cursor?

In [34]:
# 执行SQL，并返回收影响行数
effect_row = cursor.execute("select * from locations")
effect_row

23

In [20]:
# 更新数据
cursor.execute("select * from locations")

23

In [37]:
# 获取返回的数据 的描述 desc
cursor.description

(('location_id', 3, None, 11, 11, 0, False),
 ('street_address', 253, None, 40, 40, 0, True),
 ('postal_code', 253, None, 12, 12, 0, True),
 ('city', 253, None, 30, 30, 0, True),
 ('state_province', 253, None, 25, 25, 0, True),
 ('country_id', 253, None, 2, 2, 0, True))

In [36]:
cursor.fetchmany(2)  # 多条数据

((1000, '1297 Via Cola di Rie', '00989', 'Roma', 'Roma', 'IT'),
 (1100, '93091 Calle della Testa', '10934', 'Venice', None, 'IT'))

In [23]:
cursor.fetchone()

{'location_id': 1100,
 'street_address': '93091 Calle della Testa',
 'postal_code': '10934',
 'city': 'Venice',
 'state_province': None,
 'country_id': 'IT'}

In [26]:
csv.writer?

In [49]:
# 从mysql中读取数据写入csv中
def write_sql_to_file(file_name, sql, conn, with_header=True, delimiter=',',
                      quotechar='"', quoting=csv.QUOTE_NONNUMERIC, con_sscursor=False):
    cur = conn.cursor(pymysql.cursors.SSCursor) if con_sscursor else conn.cursor()
    cur.execute(sql)
    with open(file_name, 'w') as file:
        csv_writer = csv.writer(file, delimiter=delimiter, quotechar=quotechar, quoting=quoting)
        if with_header:
            header = [desc[0] for desc in cur.description]
            print(header)
            csv_writer.writerow(header)
        if con_sscursor:  # 读取大量数据
            while True:
                x = cur.fetchone()
                if x:
                    csv_writer.writerow(x)
                else:
                    break
        else:
            for x in cur.fetchall():
                csv_writer.writerow(x)
        cur.close()
    

In [50]:
write_sql_to_file('locations.csv', 'select * from locations', conn)

['location_id', 'street_address', 'postal_code', 'city', 'state_province', 'country_id']


## sql 注入

In [51]:
query = "SELECT employee_id, first_name FROM employees WHERE employee_id < {}"
# 正常查询
cursor.execute(query.format(105))


5

In [60]:
cursor.execute?

In [52]:
# sql注入
cursor.execute(query.format('105 or 1'))

110

In [54]:
# 构造预编译模板化执行sql语句，防止sql注入
cursor.execute("SELECT employee_id, first_name FROM employees WHERE employee_id < %s", (105,))

5

In [55]:
cursor.execute("SELECT employee_id, first_name FROM employees WHERE employee_id < %s", ('105 or 1',))

d:\python3.7\lib\site-packages\pymysql\cursors.py:170: Warning: (1292, "Truncated incorrect DOUBLE value: '105 or 1'")
  result = self._query(query)


5

In [13]:
# # 提交，不然无法保存新建或者修改的数据
conn.commit()

In [56]:
# 关闭游标
cursor.close()
# 关闭连接
conn.close()

## 使用上下文管理器 自动关闭连接


In [58]:
from contextlib import contextmanager

@contextmanager
def mysql(host='127.0.0.1', port=3306, user='root', passwd='', db='myemployees', charset='utf8', cur=None):
    conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db, charset=charset)
    cursor = conn.cursor(cursor=cur) if cur else conn.cursor()
    try:
        yield cursor
    finally:
        conn.commit()
        cursor.close()
        conn.close()

In [61]:
with mysql(cur=pymysql.cursors.DictCursor) as cursor:
    effect_rows = cursor.execute('select * from employees')
    print(effect_rows)
    print(cursor.description)
    print(cursor.fetchone())

110
(('employee_id', 3, None, 6, 6, 0, False), ('first_name', 253, None, 20, 20, 0, True), ('last_name', 253, None, 25, 25, 0, True), ('email', 253, None, 25, 25, 0, True), ('phone_number', 253, None, 20, 20, 0, True), ('job_id', 253, None, 10, 10, 0, True), ('salary', 5, None, 10, 10, 2, True), ('commission_pct', 5, None, 4, 4, 2, True), ('manager_id', 3, None, 6, 6, 0, True), ('department_id', 3, None, 4, 4, 0, True), ('hiredate', 12, None, 19, 19, 0, True))
{'employee_id': 100, 'first_name': 'Steven', 'last_name': 'K_ing', 'email': 'SKING', 'phone_number': '515.123.4567', 'job_id': 'AD_PRES', 'salary': 24000.0, 'commission_pct': None, 'manager_id': None, 'department_id': 90, 'hiredate': datetime.datetime(1992, 4, 3, 0, 0)}
